In [5]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
import pandas_profiling
from sklearn.preprocessing import StandardScaler
from pandas.plotting import scatter_matrix
%matplotlib inline

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve,KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve,auc
from mlxtend.classifier import StackingCVClassifier
from scipy import stats
from sklearn.preprocessing import MinMaxScaler

In [6]:
# Loading the data set into a pandas data frame

df=pd.read_csv('Train-Test-Telco-Customer-Retention.csv')

print(os.listdir("./"))

['data_preprocess.ipynb', '.DS_Store', 'LICENSE', 'NewData-Telco-Customer-Retention.csv', 'data_clean.ipynb', 'data_prep_test1.ipynb', 'Train-Test-Telco-Customer-Retention.csv', 'README.md', '.gitignore', '.git', 'WA_Fn-UseC_-Telco-Customer-Churn.csv']


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7000 non-null   object 
 1   gender            7000 non-null   object 
 2   SeniorCitizen     7000 non-null   int64  
 3   Partner           7000 non-null   object 
 4   Dependents        7000 non-null   object 
 5   tenure            7000 non-null   int64  
 6   PhoneService      7000 non-null   object 
 7   MultipleLines     7000 non-null   object 
 8   InternetService   7000 non-null   object 
 9   OnlineSecurity    7000 non-null   object 
 10  OnlineBackup      7000 non-null   object 
 11  DeviceProtection  7000 non-null   object 
 12  TechSupport       7000 non-null   object 
 13  StreamingTV       7000 non-null   object 
 14  StreamingMovies   7000 non-null   object 
 15  Contract          7000 non-null   object 
 16  PaperlessBilling  7000 non-null   object 
